# Advanced Object-Oriented Programming: Metaclasses and Class Decorators

A `class` is a "factory for objects (instances)"

What is a "factory for classes?"

In [ ]:
class MyClass1:
    pass

In [ ]:
obj = MyClass1()
type(obj)

In [ ]:
type(MyClass1)

In [ ]:
isinstance(obj, MyClass1)

In [ ]:
isinstance(MyClass1, type)

### Building classes with the `type` function

In [ ]:
MyClass16 = type('MyClass16', (MyClass1,), {})

In [ ]:
MyClass16

In [ ]:
isinstance(MyClass16, type)

In [ ]:
issubclass(MyClass16, MyClass1)

In [ ]:
MyClass16.mro()

In [ ]:
myobj16 = MyClass16()

In [ ]:
isinstance(myobj16, MyClass16)

In [ ]:
MyClass16 = type('MyClass16', (MyClass1,), {
    'training_class_attribute': 42
})

In [ ]:
MyClass16.training_class_attribute

In [ ]:
class MyClass16(MyClass1):
    training_class_attribute = 42

In [ ]:
MyClass16.training_class_attribute

In [ ]:
MyClass16.__dict__

In [ ]:
def instance_initializer(self):
    '''This will become __init__ below...'''
    print('Creating an instance')
    self.training_instance_attribute = 31337
    
MyClass17 = type('MyClass17', (MyClass1,), {
    'training_class_attribute': 42,
    '__init__': instance_initializer,
})

In [ ]:
myobj = MyClass17()

In [ ]:
myobj.training_instance_attribute

In [ ]:
myobj.training_class_attribute

In [ ]:
MyClass17.__dict__

In [ ]:
class MyClass17(MyClass1):
    "This is the class docstring"
    training_class_attribute = 42
    
    def __init__(self):
        print('Creating an instance')
        self.training_instance_attribute = 31337

In [ ]:
MyClass17.__dict__

In [ ]:
myobj.__dict__

# Introducing Metaclasses

metaclass is to class as class is to instance

In [ ]:
%%python2
def meta(name, bases, dct):
    print "Calling the metaclass"
    dct['foo'] = 'bar'
    return type(name, bases, dct)

class MyClass2(object):
    __metaclass__ = meta

print MyClass2.foo

In [ ]:
def cabbage(name, bases, dct):
    print('Inside meta, ', name, bases, dct)
    return type(name, bases, dct)

class MyClass2(metaclass=cabbage):
    a = 'foo'
    def __init__(self):
        pass


## Why oh why??

In [ ]:
!pip install marshmallow
import marshmallow as mm

```json
{
    "first": "Rick",
    "last": "Copeland",
    "email": "rick@arborian.com"
}
```

In [ ]:
class UserSchema(mm.Schema):
    first = mm.fields.Str(missing='Gutzon')
    last = mm.fields.Str(missing='Borglum')
    email = mm.fields.Str(required=True)

In [ ]:
f = mm.fields.Str(missing='Gutzon')

In [ ]:
f.name

In [ ]:
UserSchema._declared_fields

In [ ]:
UserSchema.first   # ???! definitely magic

```python
# NOT valid marshmallow
UserSchema = make_schema(
    fields=[
        Str('first', missing='Gutzon'),
        ...
    ]
)

UserSchema = make_schema(
    first=Str(missing='Gutzon'),
    ...
)
```

In [ ]:
sch = UserSchema()
sch.load({'email': 'rick@arborian.com'})

In [ ]:
mm.schema.Schema??

In [ ]:
mm.schema.SchemaMeta??

### Perverse metaclass example

In [ ]:
def make_weird_dict(name, bases, dct):
    return {
        name: value 
        for name, value in dct.items() 
        if not name.startswith('_')
    }

class Instructor(metaclass=make_weird_dict):
    name = 'Rick Copeland'
    city = 'Atlanta'

In [ ]:
Instructor

In [ ]:
type(Instructor)

### </Perverse metaclass example>

# Introducing `__new__`

In [ ]:
class NonMeta:
    def __new__(cls, *args):
        print('Calling __new__ with', cls, args)
        self = super().__new__(cls)
        self.extra_attribute = 'Victory!'
        return self
    
    def __init__(self, a, b):
        print('Calling __init__', a, b)
        self.a, self.b = a, b


In [ ]:
inst = NonMeta(1,2)

In [ ]:
inst.extra_attribute

In [ ]:
inst.a, inst.b

In [ ]:
# type(a, b, c) ==> type.__new__(type, a, b, c)
class WithAMeta(type):
    def __new__(meta, name, bases, dct):
        print('Create new class', name, 'with WithAMeta metaclass')
        print(f'meta={meta}, name={name}, bases={bases}, dct={dct}')
        dct['a'] = 5
        return super().__new__(meta, name, bases, dct)
        # return type.__new__(meta, name, bases, dct)
    

In [ ]:
class MyClass3(metaclass=WithAMeta):
    pass

In [ ]:
MyClass3.a

In [ ]:
type(MyClass3)

In [ ]:
MyClass3 = WithAMeta('MyClass3', (), {})

In [ ]:
MyClass3.a

In [ ]:
type(MyClass3)

In [ ]:
type(WithAMeta)

In [ ]:
isinstance(WithAMeta, type)

In [ ]:
issubclass(WithAMeta, type)

"Proper" metaclasses are inherited by subclasses; "function" metaclasses are not

In [ ]:
def metafun(name, bases, dct):
    print('Call metafun')
    return type(name, bases, dct)

In [ ]:
class MetaCls(type):
    def __new__(meta, name,bases, dct):
        print('Call metacls')
        return super().__new__(meta, name, bases, dct)

In [ ]:
class FunCls(metaclass=metafun):
    pass

class ClsCls(metaclass=MetaCls):
    pass

In [ ]:
class FunCls2(FunCls): pass

In [ ]:
type(FunCls)

In [ ]:
class ClsCls2(ClsCls): pass

In [ ]:
type(ClsCls)

### Metaclass trick: instance level property declaration

In [ ]:
from copy import deepcopy

class PropertyMeta(type):
    def __new__(meta, name, bases, dct):
        print('Create class', name)
        public_attributes = {
            key: value 
            for key, value in dct.items()
            if not callable(value)   # no methods
            if not key.startswith('_')   # no private attrs
        }
        dct['_public_attributes'] = public_attributes
        return super().__new__(meta, name, bases, dct)
    
    

In [ ]:
class PublicBase(metaclass=PropertyMeta):
    def __init__(self):
        for key, value in self._public_attributes.items():
            setattr(self, key, deepcopy(value))

In [ ]:
class PublicClass(PublicBase):
    a = 5
    b = 6
    c = [7, 8, 9]
    
    def amethod(self):
        print('Calling amethod!')

In [ ]:
PublicClass._public_attributes

In [ ]:
pc = PublicClass()

In [ ]:
pc.__dict__

In [ ]:
PublicClass.__dict__

In [ ]:
pc.c.append(10)

In [ ]:
pc.c

In [ ]:
PublicClass.c

(a better way is to use dataclasses from the standard lib)

### Metaclass use case: collecting properties ('declarative programming')

In [ ]:
class ImportantField:
    def __init__(self, value, name=None):
        self.value = value
        self.name = name
        
    def __repr__(self):
        return f'<Important {self.name}: {self.value}>'

In [ ]:
class DeclarativeMeta(type):
    def __new__(meta, name, bases, dct):
        print(f'DeclarativeMeta({meta!r}, {name!r}, {bases!r}, {dct!r})')
        important = []
        for k, v in dct.items():
            if isinstance(v, ImportantField):
                if v.name is None:
                    v.name = k
                important.append(v)
        dct['_important'] = tuple(important)
        return super().__new__(meta, name, bases, dct)

In [ ]:
class DeclarativeBase(metaclass=DeclarativeMeta):
    """Use regular inheritance to get the metaclass 'for free'"""

    def __repr__(self):
        l = ['<{}'.format(self.__class__.__name__)]
        for fld in self._important:
            l.append(' {}={}'.format(fld.name, fld.value))
        l.append('>')
        return ''.join(l)
    

In [ ]:
DeclarativeBase._important

In [ ]:
# Here is the "application-level" code you'd write
    
class MyClass4(DeclarativeBase):
    a = ImportantField(1)
    b = ImportantField(2)
    c = ImportantField(3, name='cfield')

In [ ]:
MyClass4._important

In [ ]:
MyClass4()

In [ ]:
MyClass4.c

In [ ]:
MyClass4.a

In [ ]:
obj = MyClass4()
obj.a

In [ ]:
obj.c

In [ ]:
obj

### Aside: metaclass inheritance

If ClassSub with metaclass MetaClassB extends ClassSuper with MetaClassA, then
MetaClassB must be a subclass of MetaClassA:

In [ ]:
# Metaclasses
class MetaClassA(type):
    pass

class MetaClassB(MetaClassA):
    pass

class MetaClassC(type):
    pass

# "Regular" classes
class ClassSuper(metaclass=MetaClassA):
    pass

class ClassSub(ClassSuper, metaclass=MetaClassC):
    pass

### Metaclass use case: class registries

In [ ]:
class RegistryMeta(type):
    _registry = {}
    
    def __new__(meta, name, bases, dct):
        cls = super().__new__(meta, name, bases, dct)
        meta._registry[name] = cls
        return cls
    
    def lookup_meta(cls, name):
        return cls._registry[name]

class RegistryBase(metaclass=RegistryMeta):
    
    @classmethod
    def lookup(cls, name):
        return cls._registry[name]
    
# RegistryBase = RegistryMeta(
#     'RegistryBase', (), {
#         'lookup': lookup_function
#     }
# )

In [ ]:
RegistryMeta._registry

In [ ]:
class Registered1(RegistryBase):
    a = ImportantField('Registered2')

class Registered2(RegistryBase):
    pass
    
class Registered3(metaclass=RegistryMeta):
    pass

In [ ]:
RegistryMeta._registry

In [ ]:
RegistryBase.lookup('Registered1')

In [ ]:
RegistryBase.lookup('Registered2')

In [ ]:
RegistryBase.lookup_meta('Registered1')

In [ ]:
RegistryBase.lookup_meta('Registered2')

In [ ]:
RegistryBase.lookup_meta('Registered3')

In [ ]:
class Employee(mm.Schema):
    name = mm.fields.Str()
    supervisor = mm.fields.Nested('Employee')

Also see `__init_subclass__`

In [ ]:
class Super:
    def __init_subclass__(cls):
        print('In init sublcass, cls == ', cls)
        
class Sub(Super):
    pass

### Metaclass trick: class operators

In [ ]:
class RegistryMeta(type):
    _registry = {}
    
    def __new__(meta, name, bases, dct):
        cls = super().__new__(meta, name, bases, dct)
        meta._registry[name] = cls
        return cls
    
    def __getitem__(cls, name):
        """Implements cls[name]"""
        return cls._registry[name]
    
    @property
    def foo(self):
        return '123'
    
    def mclass_method(cls):
        """cls is an instance of RegistyMeta, which is a class"""
        pass
    
class RegistryBase(object, metaclass=RegistryMeta):
    pass
  
#     sadly, this does not work...
#     @classmethod
#     def __getitem__(cls, name):
#         return cls._registry[name]
    
class Registered1(RegistryBase):
    def regular_method(self):
        '''self is an instance of Registered1'''
        pass
    
    @classmethod
    def class_method(cls):
        '''cls is Registered1'''
        pass

class Registered2(RegistryBase):
    pass

Properties?

In [ ]:
Registered2.foo

In [ ]:
Registered2.foo = 5

In [ ]:
RegistryBase['Registered1']

In [ ]:
RegistryBase['Registered2']

Trick: inheritance via `+`

In [ ]:
class HierMeta(type):
    
    def __add__(cls, other):
        return type(
            f'<AnonCls>({cls.__name__}, {other.__name__})', 
            (cls, other), 
            {}
        )

class HierBase(metaclass=HierMeta): pass

In [ ]:
class R1(HierBase): pass
class R2(HierBase): pass

In [ ]:
R3 = R1 + R2

In [ ]:
R3

In [ ]:
R3.mro()

In [ ]:
class Animal(HierBase): pass
class Biped(Animal): pass
class Quadruped(Animal): pass
class Monkey(Biped): pass
class Dog(Quadruped): pass
MonkeyDog = Monkey + Dog

In [ ]:
MonkeyDog

In [ ]:
MonkeyDog.mro()

In [ ]:
class ChangeNameMeta(type):
    def __new__(meta, name, bases, dct):
        dct['__qualname__'] = f'Changed{dct["__qualname__"]}'
        return super().__new__(meta, f'Changed{name}', bases, dct)

In [ ]:
class MyName(metaclass=ChangeNameMeta): pass

In [ ]:
MyName.__name__

In [ ]:
MyName

In [ ]:
MyName()

Marshmallow uses registries, as well

In [ ]:
class TreeSchema(mm.Schema):
    value = mm.fields.Str()
    left = mm.fields.Nested('TreeSchema', missing=None)
    right = mm.fields.Nested('TreeSchema', missing=None)

In [ ]:
schema = TreeSchema()

In [ ]:
schema.load({'value': 'root', 'left': {
    'value': 'left'
}})

### Class decorators: cheap substitute for metaclasses

Recall that

```python
@foo
def func(...):
    ...
```

really means

```python
def func(...):
    ...
func = foo(func)
```

In [ ]:
def decorator_factory(a, b):
    def decorator(function):
        print(f'Decorating {function} with ({a}, {b}): {id(decorator)}')
        def wrapper(*args, **kwargs):
            print('Calling decorated function', a, b)
            result = function(*args, **kwargs)
            ...
            return result
        return wrapper
    print(f'Creating decorator with args({a}, {b}): {id(decorator)}')
    return decorator

In [ ]:
deco = decorator_factory(1,2)

In [ ]:
@deco
def my_cool_function(c, d, e):
    print('cool: ', c, d, e)

In [ ]:
my_cool_function

In [ ]:
my_cool_function(10, 20, e=256)

In [ ]:
@decorator_factory(1,2)
def my_cool_function(c, d, e):
    print('cool: ', c, d, e)

In [ ]:
my_cool_function(5,6,7)

```python
@locking(the_vault)
def transfer_money(a, b, amount):
    pass
```

We can do the same with classes:

```python
@foo
class Bar:
    ...
```

means

```python
class Bar:
    ...
Bar = foo(Bar)
```

In [ ]:
def deco(cls):
    print(f'Calling deco({cls})')
    return cls

def meta(name, bases, dct):
    print(f'Calling meta({name})')
    return type(name, bases, dct)

@deco
class MyClass(metaclass=meta):
    print('In class definition')
    
print('Class body, metaclass, and decorator have all run by now')

In [ ]:
class Registry:
    
    def __init__(self):
        self._registry = {}

    def register(self, cls):
        """This is to be used as a class decorator...."""
        self._registry[cls.__name__] = cls
        return cls

    def __getitem__(self, name):
        return self._registry[name]
    
r = Registry()

In [ ]:
# "Application" code
@r.register
class Registered1:
    pass

#Registered1 = r.register(Registered1)

@r.register
class Registered2():
    pass

In [ ]:
r._registry

In [ ]:
r['Registered1']

In [ ]:
r['Registered2']

Real-world example: [Flask-RESTPlus][rest+], [Flask-SMOREST][smorest]

[rest+]: https://flask-restplus.readthedocs.io/en/stable/quickstart.html#a-minimal-api
[smorest]: https://flask-smorest.readthedocs.io/en/latest/quickstart.html

### ImportantField example

In [ ]:
class ImportantField:
    def __init__(self, value, name=None):
        self.value = value
        self.name = name
        
    def __repr__(self):
        return f'<Important {self.name}: {self.value}>'
        
        
def declarative(cls):
    print('Running declarative() decorator')
    _important = []
    for name in dir(cls):
        value = getattr(cls, name)
        if isinstance(value, ImportantField):
            if value.name is None:
                value.name = name
            _important.append(value)
    cls._important = _important
    return cls
    
@declarative
class MyClass:
    def __repr__(self):
        return f"<MyClass {' '.join(repr(i) for i in self._important)} >"
    
    a = ImportantField(1)
    b = ImportantField(2)
    c = ImportantField(3, 'cfield')

    

In [ ]:
obj = MyClass()

In [ ]:
obj

In [ ]:
MyClass._important

Class decorators are _not_ inherited:

In [ ]:
class OtherClass(MyClass):
    d = ImportantField(5)

In [ ]:
OtherClass()

Example from Barin: 
    
```python
@b.cmap(class_collection)
class MyClass(library.MySuperClass):
    a = SomeThing()
    b = SomeThing()
```

Another real-world example: [DataClasses][dataclasses]

[dataclasses]: https://docs.python.org/3/library/dataclasses.html

## Metaclass semantics

bar = Foo()

- if Foo is a class, bar is an instance
- if Foo is a metaclass, bar is a class

- `type` is a metaclass
- `object` is class
- `'foo'` is an instance (of the class `str`, which has metaclass `type`)

In [ ]:
type('foo')

In [ ]:
type(str)

In [ ]:
type(type)

But how does this work?...

In [ ]:
class Duck:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        print("Calling name's getter")
        return self._name
    
    @name.setter
    def name(self, value):
        print("Calling name's setter")
        if value.lower() in ('ed', 'bugs', 'porky'):
            raise ValueError('Inappropriate name')
        self._name = value
    #name = name.setter(name)

In [ ]:
d = Duck('Donald')

In [ ]:
d.name

In [ ]:
d.name = 'Lewey'

In [ ]:
d.name = 'ed'

# Descriptors

... or "ever wonder how `@property`, `@classmethod`, and `@staticmethod` work?"

**Descriptors** are object which contain one or more of the following magic methods, and which occur in a class body:

- `__get__(self, inst, type)` - called when the descriptor attribute is looked up (e.g. `getattr()`)
- `__set__(self, inst, value)` - called when the descriptor attribute is set (e.g. `setattr()`)
- `__delete__(self, inst)` - called when the descriptor attribute is deleted (e.g. `delattr()` or `del inst.attr`)
- `__set_name__(self, cls, name)` - called to notify the descriptor of its name within the class

In [ ]:
class MyDesc:
    
    def __get__(self, inst, cls):
        # default behavior is something like return inst.__dict__[my_own_name]
        print(f'Calling __get__({self}, {inst}, {cls})')
        if inst is None:
            return self

    def __set__(self, inst, value):
        print(f'Calling __set__({self}, {inst}, {value})')

    def __delete__(self, inst):
        print(f'Calling __delete__({self}, {inst})')
        
    def __set_name__(self, cls, name):
        print(f'Setting name of the descriptor {self} in class {cls} to {name}')

In [ ]:
the_descriptor = MyDesc()

class MyClass:
    a = the_descriptor       # a.__set_name__ is called at class creation time
    b = the_descriptor
    c = the_descriptor
    
    def __repr__(self):
        return '<Instance of MyClass>'

```python
# Psuedo-code for what's happening at class creation time
for key, value in dct.items():
    if hasattr(value, '__set_name__'):
        value.__set_name__(key)
```

In [ ]:
myobj = MyClass()

In [ ]:
MyClass.a   # invokes a.__get__(None, MyClass)

In [ ]:
myobj.a     # invoke a.__get__(myobj, MyClass)

In [ ]:
# MyClass.a = 5  # Overwrites the descriptor, so don't do this

In [ ]:
myobj.a = 20   # invokes a.__set__(myobj, 20)

In [ ]:
del myobj.a    # invokes a.__delete__(myobj)

### Descriptor "trick": shared data

In [ ]:
class SharedValue:
    
    def __init__(self, value):
        self._value = value
        
    def __get__(self, obj, typ):
        if obj is None:
            return self
        return self._value
    
    def __set__(self, obj, value):
        self._value = value
        
        
class NonsharedValue:
    def __init__(self, value):
        self._default_value = value
        self._values = {}
        
    def __get__(self, obj, typ):
        if obj is None:
            return self
        return self._values.get(id(obj), self._default_value)
    
    def __set__(self, obj, value):
        self._values[id(obj)] = value

In [ ]:
class Shared:
    a = SharedValue(123)
    b = SharedValue(456)
    c = NonsharedValue(789)
    
    def __repr__(self):
        return f'<Shared a={self.a} b={self.b} c={self.c}>'

In [ ]:
s0 = Shared()
s1 = Shared()
print(s0, s1)

In [ ]:
s0.a = 'New avalue'  # ==> Shared.__set__(self, s0, 'New avalue')
s0.c = 'New cvalue'  # ==> NonShared.__set__(self, s0, 'New cvalue')
print(s0, s1)

Let's re-implement `@property`:

In [ ]:
class myproperty:

    def __init__(self, getter, setter=None, deleter=None):
        self._getter = getter
        self._setter = setter
        self._deleter = deleter
        
    def __get__(self, inst, cls):
        print('Calling __get__')
        if inst is None:
            return self
        return self._getter(inst)
    
    def __set__(self, inst, value):
        print('Calling __set__')
        if self._setter is None:
            raise TypeError('value is read-only')
        self._setter(inst, value)
        
    def __delete__(self, inst):
        print('Calling __delete__')
        if self._deleter is None:
            raise TypeError('value is undeleteable')
        self._deleter(inst)
        
    def setter(self, setter):
        """Decorator to add a setter"""
        self._setter = setter
        return self
    
    def deleter(self, deleter):
        """Decorator to add a deleter"""
        self._deleter = deleter
        return self

In [ ]:
class Foo:   # Foo()    Foo(object)
    
    @myproperty
    def bar(self):
        print('Calling getter for bar')
        return 'barval'
    #bar = myproperty(bar)
    
    @bar.setter
    def bar(self, value):
        print('Calling setter for bar')
    # bar is an instance of myproperty on this line
        
#     _tmp0 = bar.setter
#     def bar(self, value):
#         print('Calling the setter for bar')
#     bar = _tmp0(bar)
        
    @bar.deleter
    def bar(self):  # del foo.bar
        print('Calling deleter for bar')
        
foo = Foo()

In [ ]:
Foo.__dict__['bar']

In [ ]:
Foo.bar

In [ ]:
foo.bar

In [ ]:
foo.bar = 10

In [ ]:
foo.bar

In [ ]:
del foo.bar

In [ ]:
Foo.bar

In [ ]:
property??

## Descriptor types

- A **data descriptor** is a descriptor that defines both `__get__` and `__set__`
- A **non-data descriptor** is a descriptor that defines only `__get__`

> Data and non-data descriptors differ in how overrides are calculated with respect to entries in an instance’s dictionary. If an instance’s dictionary has an entry with the same name as a data descriptor, the data descriptor takes precedence. If an instance’s dictionary has an entry with the same name as a non-data descriptor, the dictionary entry takes precedence.

So order of precedence in attribute access is:

- data descriptor
- instance `__dict__`
- non-data descriptor

In [ ]:
class Foo(): 
    pass

foo = Foo()
foo.a = 5

In [ ]:
foo.__dict__

In [ ]:
class DataDescriptor():
    
    def __get__(self, obj, typ):
        if obj is None:
            return self
        return 'data descriptor value'
    
    def __set__(self, obj, value):
        # Just make it a read-only data descriptor
        raise TypeError('read-only property')

In [ ]:
class NonDataDescriptor():
    
    def __get__(self, obj, typ):
        if obj is None:
            return self
        return 'non-data descriptor value'
        

In [ ]:
class MyClass():
    data = DataDescriptor()
    nondata = NonDataDescriptor()

In [ ]:
obj = MyClass()
obj.__dict__.update(
    data='instance data',
    nondata='instance nondata'
)

Data descriptors have precedence over instance data:

In [ ]:
obj.data

Instance data has precendence over non-data descriptors:

In [ ]:
obj.nondata

In [ ]:
obj.nondata = 'something else'

In [ ]:
obj.nondata

If we delete it from the instance dict, however, it _will_ invoke the non-data descriptor's `__get__` method

In [ ]:
del obj.nondata

In [ ]:
obj.nondata

## Descriptor use case: cached property

In [ ]:
class cached_property:  # "reify" is another name for this
    
    def __init__(self, getter):
        self._getter = getter
        self._name = None
        
    def __set_name__(self, cls, name):
        self._name = name
    
    def __get__(self, inst, cls):
        if inst is None:
            return self
        value = self._getter(inst)
        setattr(inst, self._name, value) # put the value in the instance __dict__
        # alternatively, inst.__dict__[name] = value
        return value
    
#     def __set__(self, obj, value):
#         obj.__dict__[self._name] = value

In [ ]:
class CachedExample:
    
    @cached_property
    def prop(self):
        print('Calculating CachedExample.prop')
        return 42


In [ ]:
ce = CachedExample()

In [ ]:
ce.prop # adds prop to ce.__dict__

In [ ]:
ce.prop  # Since prop is non-data descriptor, it is not even accessed here

In [ ]:
import sys
sys.version_info

In "real life", this is implemented in `functools.cached_property` (in Python 3.8+)

```python
import functools
help(functools.cached_property)
```

In [ ]:
import functools
help(functools.cached_property)

In [ ]:
functools.cached_property??

Open [Advanced OOP Lab](./advanced-oop-lab.ipynb)

In [ ]:
class SchemaMeta(type):
    def __new__(meta, name, bases, dct):
        return super().__new__(meta, name, bases, dct)
    
class Schema(metaclass=SchemaMeta):
    pass

In [ ]:
def foo(x: SchemaMeta) -> None:
    pass

In [ ]:
foo(Cls)

In [ ]:
import typing